# Series Temporales

Las series temporales son conjuntos de datos que están ordenados en función del tiempo. En otras palabras, representan observaciones o mediciones que se han recopilado en intervalos regulares de tiempo, como días, meses, horas, etc

Es muy importante tener definidas las columnas de fechas para trabajar con series temporales, al importar tu archivo a pandas, puedes indicarle que columnas necesitas que usen el formato fecha con el parametro parse_date


In [ ]:
import pandas as pd

data = pd.read_csv('D:/Tripleten/datasets/energy_consumption.csv', index_col=[0], parse_dates=[0]) #parse dates transforma las columnas indicadas a formato fecha

data.sort_index(ascending=True, inplace=True)
print(data.index.is_monotonic_increasing)
# print(data.info())

Se pueden realizar busquedas solo especificando la comparacion entre años `data['2016':'2018]` o años y meses `data['2016-01':'2016-12']  `

In [ ]:
data_2018 = data['2018-01':'2018-12'].resample('ME').mean().plot(title='Media de Consumo electrico por Mes')

# Media móvil

La media móvil o promedio móvil es un método para suavizar los datos en una serie temporal.  El método consiste en encontrar los valores menos susceptibles a fluctuaciones, es decir, la media aritmética.

En python se establece a través de la funcion rolling() en donde se especifica el número de media aritmeticas a tomar y se complementa con la función deseada (suma, media, maximo, desviación std, etc.)



In [ ]:
import matplotlib.pyplot as plt

mobile_mean =  data['2015':'2018'].resample('QE').sum()
mobile_mean['std'] = mobile_mean.rolling(3).std()

mobile_mean.plot(title='Serie Temporal')
mobile_mean['std'].plot(color='orange')
plt.legend(['MW_Consumption', 'Standard Deviation'])
plt.show()

# Tendencias y Estacionalidad

- Tendencia: Una tendencia es un cambio ligero del valor medio de la serie sin repetir patrones. Por ejemplo, el incremento anual en la venta de boletos de avión.
- Estacionalidad: significa patrones que se repiten de forma cíclica en una serie temporal. Por ejemplo, el crecimiento de las ventas de boletos de avión cada verano.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import pandas as pd
import matplotlib.pyplot as plt


data = pd.read_csv('D:/Tripleten/datasets/energy_consumption.csv', index_col=[0], parse_dates=[0]) #parse dates transforma las columnas indicadas a formato fecha
data.sort_index(ascending=True, inplace=True)
data = data['2013':'2018'].resample('1ME').sum()
decomposed = seasonal_decompose(data)

plt.figure(figsize=(10, 10))

plt.subplot(311)
decomposed.trend.plot(ax=plt.gca())
plt.title('Tendencia')
plt.subplot(312)
decomposed.seasonal['2013':'2014'].plot(ax=plt.gca()) # Es posible filtrar con los datos ya procesados.
plt.title('Estacionalidad')
plt.subplot(313)
decomposed.resid.plot(ax=plt.gca())
plt.title('Residuales')
plt.tight_layout()
plt.show()

# Diferencias de series temporales

Diferencias de series temporales es una secuencia de diferencias entre elementos vecinos de una serie temporal (es decir, el valor anterior se resta del siguiente).

El método shift() se usa para encontrar las diferencias de series temporales. Todos los valores se desplazan un paso hacia adelante a lo largo del eje de tiempo:

In [ ]:
data = pd.Series([0.5, 0.7, 2.4, 3.2])

data -= data.shift(fill_value=0)

plt.plot(data)
plt.show()

# Pronóstico de series temporales

El objetivo del pronóstico de series temporales es desarrollar un modelo que prediga los valores futuros de una serie temporal con base en datos anteriores.

El periodo en el futuro para el que se prepara el pronóstico se conoce como horizonte de pronóstico. Para los ejercicios de este capítulo usaremos un horizonte de un paso. Es muy importante que al entrenar un modelo dividamos los datos en entrenamiento y test, sin embargo a diferencia de otros modelos, nuestros datos no deberán de ser mezclados aleatoriamente, ya que arruinaria la continuidad de nuestra serie temporal, en su lugar aplicaremos el parametro `shuffle=False`

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.Series([0.1, 0.5, 2.3, 1.2, 1.5])
train, test = train_test_split(data, shuffle=False, test_size=1/5) # Desactivamos shuffle para series temporales
print('Conjunto de entrenamiento:')
print(train)
print('Conjunto de prueba:')
print(test)

In [ ]:
data.index.dayofweek

# Exactitud del pronóstico


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


data = pd.read_csv(
    'D:/Tripleten/datasets/energy_consumption.csv', index_col=[0], parse_dates=[0]
)
data.sort_index(inplace=True)
data = data.resample('1D').sum()

train, test = train_test_split(data, shuffle=False, test_size=0.2)

print('Consumo medio diario de energía:', test['PJME_MW'].median())

pred_previous = test.shift()  # Con esto se reemplazan los valores al día siguiente y el primer valor se queda en 0
pred_previous.iloc[0] = train.iloc[-1] # Con esto traemos el último día de train y se lo ponemos al primer día de test
print('EAM:', mean_absolute_error(test, pred_previous))

# Creación de características

Caracteristicas de calendario

In [ ]:
data = pd.read_csv('D:/Tripleten/datasets/energy_consumption.csv', index_col=[0], parse_dates=[0])
data.sort_index(inplace=True)
data = data.resample('1D').sum()

# esta característica contiene años como valores numéricos
data['año'] = data.index.year

# esta característica contiene días de la semana como valores numéricos
data['díadelasemana'] = data.index.dayofweek

# esta característica contiene días de la semana como valores numéricos
data['día'] = data.index.day

# esta característica contiene días de la semana como valores numéricos
data['mes'] = data.index.month

print(data.sample(8))

Características de desfase

Los valores anteriores en la serie temporal te dirán si la función x(t) aumentará o disminuirá. Vamos a usar la función shift() para obtener los valores de desfase:

In [ ]:
data['lag_1'] = data['PJME_MW'].shift(1, fill_value=0) # En caso de que queramos rellenar
data['lag_2'] = data['PJME_MW'].shift(2)
data['lag_3'] = data['PJME_MW'].shift(3)

data.head(6)

Ahora un ejercicio aplicando todo


In [ ]:
import pandas as pd
import numpy as np


data = pd.read_csv(
    'D:/Tripleten/datasets/energy_consumption.csv', index_col=[0], parse_dates=[0]
)
data.sort_index(inplace=True)
data = data.resample('1D').sum()

def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek
    
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['PJME_MW'].shift(lag)

    data['rolling_mean']= data['PJME_MW'].shift().rolling(rolling_mean_size).mean()

make_features(data, 4, 4)
data.head()

# Series Temporales con Machine Learning

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error


data = pd.read_csv(
    'D:/Tripleten/datasets/energy_consumption.csv', index_col=[0], parse_dates=[0]
)
data.sort_index(inplace=True)
data = data.resample('1D').sum()

def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['dayofweek'] = data.index.dayofweek

    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['PJME_MW'].shift(lag)

    data['rolling_mean'] = (
        data['PJME_MW'].shift().rolling(rolling_mean_size).mean()
    )


make_features(data, 6, 10)

train, test = train_test_split(data, shuffle=False, test_size=0.2)
train = train.dropna()

X_train = train.drop(columns=['PJME_MW'])
y_train = train['PJME_MW']

X_test = test.drop(columns=['PJME_MW'])
y_test = test['PJME_MW']

model = LinearRegression()
model.fit(X_train,y_train)

X_predict = model.predict(X_train)
y_predict = model.predict(X_test)

MAE_train = mean_absolute_error(y_train,X_predict)
MAE_test = mean_absolute_error(y_test,y_predict)

print("EAM para el conjunto de entrenamiento:", MAE_train)
print("EAM para el conjunto de prueba:", MAE_test)

EAM para el conjunto de entrenamiento: 33620.831381001844
EAM para el conjunto de prueba: 36776.40327032028


In [2]:
data

,PJME_MW,year,month,day,dayofweek,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,rolling_mean
Datetime,,,,,,,,,,,,
2002-01-01,714857.0,2002,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-01-02,822277.0,2002,1,2,2,714857.0,NaN,NaN,NaN,NaN,NaN,NaN
2002-01-03,828285.0,2002,1,3,3,822277.0,714857.0,NaN,NaN,NaN,NaN,NaN
2002-01-04,809171.0,2002,1,4,4,828285.0,822277.0,714857.0,NaN,NaN,NaN,NaN
2002-01-05,729723.0,2002,1,5,5,809171.0,828285.0,822277.0,714857.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-30,790978.0,2018,7,30,0,771910.0,827898.0,916596.0,917308.0,889492.0,921189.0,843245.9
2018-07-31,828938.0,2018,7,31,1,790978.0,771910.0,827898.0,916596.0,917308.0,889492.0,838613.2
2018-08-01,941539.0,2018,8,1,2,828938.0,790978.0,771910.0,827898.0,916596.0,917308.0,851115.6


In [1]:
#adfuller() #para ver si tiene corrrelacion


# df[].autocorr(lag=1)
# df[].autocorr(lag=2)
# df[].autocorr(lag=3)


decompose = seasonal_compose(df[passengers], model= additive, period= 7) #el periodo son los cortes que queremos hacer(cada 7 meses) # Si los residuos estan en 1 esta saludable
# Si los datos estan alrededor del 0 estan saludables.

decompose = seasonal_compose(df[passengers], model= multiplicative , period= 7) # Si los residuos estan en 1 esta saludable

# from pmdarima import auto_arima (((xxx moving avergare)))
# sarimax modelo 2
#prophet python/